In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Caricamento dataset
df = pd.read_csv("common_passwords.csv")
fig = px.histogram(df, x="length", nbins=20, opacity=0.85,
                   title="🔐 Distribuzione della Lunghezza delle Password",
                   template="plotly_dark",
                   color_discrete_sequence=["deepskyblue"])
fig.update_layout(
    plot_bgcolor="#050b18",
    paper_bgcolor="#050b18",
    font=dict(color="white"),
    xaxis_title="Lunghezza",
    yaxis_title="Frequenza"
)
fig.show()
df_bar = df.groupby("length")[["num_chars", "num_digits", "num_special"]].mean().reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(name='Lettere', x=df_bar["length"], y=df_bar["num_chars"],
                     marker_color='lightskyblue'))
fig.add_trace(go.Bar(name='Cifre', x=df_bar["length"], y=df_bar["num_digits"],
                     marker_color='royalblue'))
fig.add_trace(go.Bar(name='Speciali', x=df_bar["length"], y=df_bar["num_special"],
                     marker_color='mediumblue'))

fig.update_layout(
    barmode='stack',
    title="🧱 Composizione media dei caratteri per lunghezza",
    xaxis_title="Lunghezza della password",
    yaxis_title="Conteggio medio per tipo",
    template="plotly_dark",
    plot_bgcolor="#050b18",
    paper_bgcolor="#050b18",
    font=dict(color="white")
)
fig.show()
fig = px.scatter(df, x="length", y="num_chars",
                 title="🔡 Lunghezza vs Numero di Lettere",
                 template="plotly_dark",
                 color_discrete_sequence=["cyan"],
                 opacity=0.4)

fig.update_layout(
    plot_bgcolor="#050b18",
    paper_bgcolor="#050b18",
    font=dict(color="white"),
    xaxis_title="Lunghezza",
    yaxis_title="Numero di lettere"
)
fig.show()


IDEE GPT

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Caricamento dataset
df = pd.read_csv("common_passwords.csv")
# 📊 Istogramma della lunghezza
fig = px.histogram(df, x="length", nbins=20, opacity=0.85,
                   title="🔐 Distribuzione della Lunghezza delle Password",
                   template="plotly_dark", color_discrete_sequence=["deepskyblue"])
fig.update_layout(plot_bgcolor="#050b18", paper_bgcolor="#050b18", font=dict(color="white"),
                  xaxis_title="Lunghezza", yaxis_title="Frequenza")
fig.show()
# 🧱 Barre impilate: tipo di carattere
df_bar = df.groupby("length")[["num_chars", "num_digits", "num_special"]].mean().reset_index()
fig = go.Figure()
fig.add_trace(go.Bar(name='Lettere', x=df_bar["length"], y=df_bar["num_chars"], marker_color='lightskyblue'))
fig.add_trace(go.Bar(name='Cifre', x=df_bar["length"], y=df_bar["num_digits"], marker_color='royalblue'))
fig.add_trace(go.Bar(name='Speciali', x=df_bar["length"], y=df_bar["num_special"], marker_color='mediumblue'))
fig.update_layout(barmode='stack',
    title="🧱 Composizione media dei caratteri per lunghezza",
    xaxis_title="Lunghezza", yaxis_title="Media per tipo",
    template="plotly_dark", plot_bgcolor="#050b18", paper_bgcolor="#050b18",
    font=dict(color="white"))
fig.show()



In [ ]:
# 🔢 Istogramma: solo cifre, lunghezza ≤6
df_simple = df[(df["length"] <= 6) & (df["num_digits"] == df["length"])]
fig = px.histogram(df_simple, x="length", nbins=10,
                   title="🔢 Password Corte solo Cifre (≤6)",
                   template="plotly_dark", color_discrete_sequence=["red"])
fig.update_layout(plot_bgcolor="#050b18", paper_bgcolor="#050b18", font=dict(color="white"))
fig.show()
# 🎻 Violin plot: caratteri speciali
fig = px.violin(df, y="num_special", box=True, points="outliers",
                title="🎻 Distribuzione Caratteri Speciali",
                template="plotly_dark", color_discrete_sequence=["orange"])
fig.update_layout(plot_bgcolor="#050b18", paper_bgcolor="#050b18", font=dict(color="white"))
fig.show()
# 🌲 Treemap: password con ≤1 tipo di carattere
df["type_count"] = ((df["num_chars"] > 0).astype(int) +
                    (df["num_digits"] > 0).astype(int) +
                    (df["num_special"] > 0).astype(int))
simple = df[df["type_count"] <= 1]
top_simple = simple["password"].value_counts().nlargest(20).reset_index()
top_simple.columns = ["password", "count"]
fig = px.treemap(top_simple, path=["password"], values="count",
                 title="🌲 Treemap Password con ≤1 Tipo di Carattere",
                 template="plotly_dark", color="count", color_continuous_scale="Blues")
fig.update_layout(plot_bgcolor="#050b18", paper_bgcolor="#050b18")
fig.show()


In [ ]:
# 📦 IMPORT E DATI
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv("common_passwords.csv")

# 📌 CLASSIFICAZIONE FORTE / DEBOLE
def classify_strength(row):
    score = 0
    score += row['length'] >= 8
    score += row['num_special'] > 0
    score += row['num_upper'] > 0
    score += row['num_digits'] > 0
    return 'Forte' if score >= 3 else 'Debole'

df['strength'] = df.apply(classify_strength, axis=1)
# 📊 Prepara i dati
group = df.groupby(['length', 'strength']).size().reset_index(name='count')
total = group.groupby('length')['count'].transform('sum')
group['percent'] = group['count'] / total * 100

# 📉 Plot
fig = px.bar(group, x='length', y='percent', color='strength',
             title="🟩 Percentuale di Password Forti/Deboli per Lunghezza",
             template='plotly_dark',
             color_discrete_map={'Forte': 'deepskyblue', 'Debole': 'crimson'})

fig.update_layout(
    plot_bgcolor='#050b18',
    paper_bgcolor='#050b18',
    font=dict(color='white'),
    yaxis_title="% sul totale per lunghezza",
    xaxis_title="Lunghezza"
)
fig.show()
# 🌞 Crea colonne categoriche
df['has_special'] = df['num_special'].apply(lambda x: '✓ simboli' if x > 0 else 'no simboli')
df['length_bin'] = pd.cut(df['length'], bins=[0,6,9,20], labels=['corta','media','lunga'])

fig = px.sunburst(df, path=['has_special', 'length_bin', 'strength'],
                  title="🌞 Gerarchia: Simboli → Lunghezza → Forza Password",
                  template="plotly_dark",
                  color='strength',
                  color_discrete_map={'Forte': 'lightgreen', 'Debole': 'orangered'})

fig.update_layout(
    plot_bgcolor='#050b18',
    paper_bgcolor='#050b18',
    font=dict(color='white')
)
fig.show()
# 🔥 Crea matrice
heatmap_data = df.groupby(['num_digits', 'num_special']).size().reset_index(name='count')
heatmap_data = heatmap_data.pivot(index='num_special', columns='num_digits', values='count').fillna(0)

# 🔥 Heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='Viridis'))

fig.update_layout(
    title='🔥 Frequenza Password per Num. Cifre e Simboli',
    xaxis_title='Numero Cifre',
    yaxis_title='Numero Simboli',
    template='plotly_dark',
    plot_bgcolor='#050b18',
    paper_bgcolor='#050b18',
    font=dict(color='white')
)
fig.show()



top 10 pwd

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Carica i dati
df = pd.read_csv("common_passwords.csv")

# Top 10 password
top10 = df["password"].value_counts().nlargest(10).reset_index()
top10.columns = ["Password", "Frequenza"]
top10["Posizione"] = top10.index + 1

# Barra di supporto per la visualizzazione
max_val = top10["Frequenza"].max()
top10["Bar"] = top10["Frequenza"] / max_val * 100

# Tabella dinamica con barra integrata
fig = go.Figure(data=[
    go.Table(
        header=dict(
            values=["<b>Pos</b>", "<b>Password</b>", "<b>Frequenza</b>", "<b>Visuale</b>"],
            fill_color="#1f2a39",
            font=dict(color='white', size=14),
            align="left"
        ),
        cells=dict(
            values=[
                top10["Posizione"],
                top10["Password"],
                top10["Frequenza"],
                ["▮" * int(v // 5) for v in top10["Bar"]]  # mini-bar con unicode
            ],
            fill_color="#0b101a",
            font=dict(color='lightgray', size=13),
            align="left",
            height=28
        )
    )
])

fig.update_layout(
    title="📋 Le 10 Password Più Usate – Elenco Interattivo",
    template="plotly_dark",
    margin=dict(l=20, r=20, t=60, b=20),
    paper_bgcolor="#050b18",
    plot_bgcolor="#050b18",
    font=dict(color="white", family="Courier New")
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Carica dataset e posizione implicita
df = pd.read_csv("common_passwords.csv")
df = df.reset_index().rename(columns={"index": "Posizione"})
df["Posizione"] += 1

# Seleziona top 10
top10 = df.head(10).copy()

# Palette a impatto visivo (blu → turchese → ghiaccio)
row_colors = [
    "#2196f3", "#1e88e5", "#1976d2", "#1565c0", "#0d47a1",
    "#0c3c91", "#0b327f", "#092a6e", "#081f5a", "#06184a"
]

# Aggiungi lunghezza per valore comparativo
top10["Lunghezza"] = top10["length"]

# Crea tabella
fig = go.Figure(data=[
    go.Table(
        header=dict(
            values=["<b>Posizione</b>", "<b>Password</b>", "<b>Lunghezza</b>"],
            fill_color="#0f1c2f",
            line_color='gray',
            font=dict(color='white', size=16, family="Courier New"),
            align="left",
            height=42
        ),
        cells=dict(
            values=[
                top10["Posizione"],
                top10["password"],
                top10["Lunghezza"]
            ],
            fill_color=[row_colors, row_colors, row_colors],
            line_color='darkgray',
            font=dict(color='white', size=15, family="Courier New"),
            align="left",
            height=38
        )
    )
])

# Layout
fig.update_layout(
    title="🔐 Top 10 Password Più Usate – Evidenza Visuale",
    title_font_size=20,
    template="plotly_dark",
    paper_bgcolor="#050b18",
    plot_bgcolor="#050b18",
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carica il dataset
df = pd.read_csv("common_passwords.csv")

# Prepara i dati top 20
top20 = df.head(20).copy()
top20["Posizione"] = range(1, 21)

# === 1. Bar Chart Orizzontale (Top 20 Password) ===
plt.figure(figsize=(10, 8))
sns.barplot(y=top20["password"], x=top20["length"], palette="Blues_d")
plt.title("🔐 Top 20 Password Più Comuni – Lunghezza", fontsize=14)
plt.xlabel("Lunghezza")
plt.ylabel("Password")
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# === 2. Tabella Statica (Matplotlib) ===
fig, ax = plt.subplots(figsize=(10, 8))
ax.axis('tight')
ax.axis('off')

# Prepara dati per tabella
table_data = [
    ["Posizione", "Password", "Lunghezza"]
] + list(top20[["Posizione", "password", "length"]].values)

# Crea tabella
table = ax.table(
    cellText=table_data,
    colLabels=None,
    cellLoc='left',
    loc='center',
    colWidths=[0.1, 0.5, 0.2]
)

# Stile tabella
table.auto_set_font_size(False)
table.set_fontsize(13)
table.scale(1.2, 1.5)

# Colore header
for i in range(3):
    table[(0, i)].set_facecolor("#2c3e50")
    table[(0, i)].set_text_props(color="white", weight='bold')

plt.title("📋 Top 20 Password – Tabella Statica", fontsize=16, pad=20)
plt.tight_layout()
plt.show()
